In [259]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics import classification_report


In [260]:
df = pd.read_csv("dataset_5.csv")

df.columns

Index(['Unnamed: 0', 'date', 'id', 'meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms', 'Target',
       'time_window_sumappCat.office', 'time_window_summood',
       'time_window_countappCat.office', 'time_window_sumcircumplex.valenc

In [261]:
from sklearn.model_selection import train_test_split
np.random.seed(42)


# Splitting the data into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# First, split into train and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(df[['meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms','time_window_sumappCat.office', 'time_window_summood',
       'time_window_countappCat.office', 'time_window_sumcircumplex.valence',
       'time_window_meancircumplex.valence']], (df['handmatige_bin']*0.1), test_size=test_ratio, random_state=42)

# Then, split train into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# Display the shape of each set
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (881, 40)
Validation set shape: (189, 40)
Test set shape: (189, 40)


In [262]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.layers import Dropout
import tensorflow as tf

tf.random.set_seed(42)


# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_size = X_train.shape[1]

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.1, verbose=1)



Epoch 1/100
50/50 [==============================] - 2s 10ms/step - loss: 0.0688 - val_loss: 0.0212
Epoch 2/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0316 - val_loss: 0.0137
Epoch 3/100
50/50 [==============================] - 0s 3ms/step - loss: 0.0176 - val_loss: 0.0058
Epoch 4/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0116 - val_loss: 0.0057
Epoch 5/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0088 - val_loss: 0.0045
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0085 - val_loss: 0.0041
Epoch 7/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0070 - val_loss: 0.0040
Epoch 8/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0058 - val_loss: 0.0038
Epoch 9/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0055 - val_loss: 0.0037
Epoch 10/100
50/50 [==============================] - 0s 4ms/step - loss: 0.0049 - val_loss: 0.0038
Epoch 11

In [263]:

predictions = model.predict(X_test)
predictions = predictions*10
predictions = predictions.round()
y_test = y_test*10

report = classification_report(y_test, predictions)

print("Classification Report:")
print(report)

6/6 [==============================] - 0s 3ms/step
Classification Report:
              precision    recall  f1-score   support

         0.0       0.50      0.03      0.06        29
         1.0       0.66      0.96      0.78       120
         2.0       0.54      0.17      0.26        40

    accuracy                           0.65       189
   macro avg       0.57      0.39      0.37       189
weighted avg       0.61      0.65      0.56       189



In [264]:
from sklearn.model_selection import train_test_split

# Splitting the data into train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15



# First, split into train and test sets
# Set bin or handmatife_bin to select for training on the uniform or manual bins

X_train_val, X_test, y_train_val, y_test = train_test_split(df[['meancircumplex.arousal',
       'meancircumplex.valence', 'countappCat.builtin',
       'countappCat.communication', 'countappCat.entertainment',
       'countappCat.finance', 'countappCat.game', 'countappCat.office',
       'countappCat.other', 'countappCat.social', 'countappCat.travel',
       'countappCat.unknown', 'countappCat.utilities', 'countappCat.weather',
       'countcall', 'countcircumplex.arousal', 'countcircumplex.valence',
       'countscreen', 'countsms', 'sumactivity', 'sumappCat.builtin',
       'sumappCat.communication', 'sumappCat.entertainment',
       'sumappCat.finance', 'sumappCat.game', 'sumappCat.office',
       'sumappCat.other', 'sumappCat.social', 'sumappCat.travel',
       'sumappCat.unknown', 'sumappCat.utilities', 'sumappCat.weather',
       'sumcall', 'sumscreen', 'sumsms','time_window_sumappCat.office', 'time_window_summood',
       'time_window_countappCat.office', 'time_window_sumcircumplex.valence',
       'time_window_meancircumplex.valence']], (df['handmatige_bin']).round(0), test_size=test_ratio, random_state=42)

# Then, split train into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# Display the shape of each set
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (881, 40)
Validation set shape: (189, 40)
Test set shape: (189, 40)


In [265]:
print(df['handmatige_bin'].min())
print(df['handmatige_bin'].max())

0
2


In [266]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import metrics





# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')  # Output layer with 10 units for 10 classes and softmax activation
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.1, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
y_pred = model.predict(X_test)

y_pred_new = []
for i in y_pred:
    max_val = max(i)
    for idx, j in enumerate(i):
        if j == max_val:
            y_pred_new.append(idx)

report = classification_report(y_test, y_pred_new)

print("Classification Report:")
print(report)


Epoch 1/100
50/50 [==============================] - 1s 8ms/step - loss: 1.0002 - accuracy: 0.5480 - val_loss: 0.8738 - val_accuracy: 0.6404
Epoch 2/100
50/50 [==============================] - 0s 4ms/step - loss: 0.9145 - accuracy: 0.6111 - val_loss: 0.8564 - val_accuracy: 0.6404
Epoch 3/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8971 - accuracy: 0.6086 - val_loss: 0.8358 - val_accuracy: 0.6404
Epoch 4/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8834 - accuracy: 0.6048 - val_loss: 0.8305 - val_accuracy: 0.6517
Epoch 5/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8799 - accuracy: 0.6187 - val_loss: 0.8135 - val_accuracy: 0.6404
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8535 - accuracy: 0.6338 - val_loss: 0.8103 - val_accuracy: 0.6629
Epoch 7/100
50/50 [==============================] - 0s 7ms/step - loss: 0.8508 - accuracy: 0.6427 - val_loss: 0.8140 - val_accuracy: 0.6404
Epoch 8/100
5

In [267]:
y_pred_new = []
for i in y_pred:
    max_val = max(i)
    for idx, j in enumerate(i):
        if j == max_val:
            y_pred_new.append(idx)

print(y_pred_new)

[1, 2, 1, 1, 2, 0, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1]


In [268]:
# Train a decision tree

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


# Define the decision tree classifier
tree_clf = DecisionTreeClassifier(random_state=42, loss= )

# Train the decision tree classifier
tree_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = tree_clf.predict(X_test)


report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

SyntaxError: invalid syntax (3107735736.py, line 10)

In [ ]:
# Train an SVM

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Define the multiclass SVM classifier
svm_clf = SVC(kernel='poly', decision_function_shape='ovr')  # One-vs-Rest (OvR) strategy for multiclass classification

# Train the SVM classifier
svm_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_clf.predict(X_test)


report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        29
           1       0.68      0.93      0.79       120
           2       0.54      0.33      0.41        40

    accuracy                           0.66       189
   macro avg       0.41      0.42      0.40       189
weighted avg       0.55      0.66      0.59       189



C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\tomsl\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)